# Overview

## Communication with SPM

The building block for all workflows is the **spm_control()** function, which mimics instructions from human operators and unifies the interaction with SPM controller.

* It's a wrapper function of **"write_spm()"** and **"read_spm()"**, which can be customized for any SPM controllers (depending on whether or not your controllers support API, it may need extra hardwares to realize the communication). 
* For Asylum Research controller and open-source controller, it requires no extra hardware to run this library
* This function takes three inputs:
    * action: the SPM instructions in hyper-languages (see examples below)
    * value: the value associated with the instructions. For buttons, there is no need of value input.
    * wait: the sleep time after the action execution is done.
    
___Examples:___
```Python
import aespm as ae

# Change the scan rate to 1 Hz
ae.spm_control('ScanRate', wait=0.5)
```

```Python
# Wait time is required to wait until the operation is done.
ae.spm_control('DownScan', wait=1.5)
```

## Data file I/O

__For AR controllers:__

Currently, we rely on monitoring the file numbers in the data saving folder to tell if the measurement (image/spec) has finished.

```Python
# The workflow will wait until the number of file has changes in the save_folder.
ae.spm_control('DownScan', wait=1.5)
ae.check_file_number(path=save_folder)
```

```Python
# The following codes read the most recently modified/created file from the given folder:
fname = ae.get_files(folder=save_folder)[0]
img = ae.ibw_read(fname)
```

__For open-source controller:__

To be written by Marcos.

## ___Experiment___ object

**Experiment** object is the preferred way to construct a workflow.

1. It offers a way to integrate repeating operations into custom functions, and then you can attach custom functions as new methods of the experiment object easily.
2. The modular assemble of repeating operations into functional blocks makes workflows easier to read and organize, and makes it easier to debug -- if you can make sure each modular blocks function well, then the whole workflow should work without problems.
3. It offers a **log** mthod to log all the operations, parameter changes and custom functions.
4. It offers a **param** attribute to keep track of all important parameters and intermediate results.
5. It handles all the SSH connections automatically under the hood!

**Experiment.execute()**

This is a wrapper of spm_control: it accepts the same inputs:

* action: the SPM instructions in hyper-languages (see examples below)
* value: the value associated with the instructions. For buttons, there is no need of value input.
* wait: the sleep time after the action execution is done.

```Python
exp.execute('DownScan', wait=1.5)
```

**Experiment.execute_sequence()**

This is the building block for the custom functions, as it accepts a list of actions and execute them sequentially.

* operation: a list of actions together with corresponding values and wait time.

```Python
action_list = [
    ['ChangeName', fname, None], # Change file names
    ['DownScan', None, 1.5],     # Start a down scan
    ['check_files', None, None], # Pause the workflow until the number of files changed in the save_folder 
]

exp.execute_sequence(operation=action_list)
```

**Experiment.add_func()**

This is the method to attach a custom function to the experiment object. After use this function, the custom function can be called by its name as action in Experiment.execute() and Experiment.execute_sequence():

```Python
def ac_scan(self, fname):
    action_list = [
        ['ChangeName', fname, None], # Change file names
        ['DownScan', None, 1.5],     # Start a down scan
        ['check_files', None, None], # Pause the workflow until number of files change in the save_folder 
    ]

    self.execute_sequence(operation=action_list)
    img = ae.ibw_read(ae.get_files(folder=self.folder)[0])
    
    return img

exp.add_func(ac_scan)
```

Then this custom function can be called in three ways:

```Python
# Preferred way: call it the same way as default actions
img = exp.execute('ac_scan', value='NewImage')

# Put it in the operation sequence list
action_list = [['ac_scan', 'NewImage', None], ['Stop', None, None]]
img = exe.execute_sequence(operation=action_list)

# Call as a method directly
img = exp.ac_scan('ac_scan', value='NewImage')
```

Note that the values for custom functions must be included in a list if there are multiple positional inputs or one or more of the inputs are lists.

Let's use a complicated example to show why we need to use contain the value inputs for custom functions.

```Python
# There is a default move_tip() function in aespm. 
# Here we define a new move_probe() function for demo.
# Pass keyword arguments as a dict in the end of the operation list. 
def move_probe(self, r, angle, v0, s0):
    '''
    r: displacement vector [rx, ry]
    angle: scan angle of the current topography map
    v0: the piezo voltage at the old probe position
    s0: sensitivity of LVDT [sx, sy] for x and y
    '''
    ae.move_tip(r=r, angle=angle, v0=v0, s=s0)
    
exp.add_func(move_probe)

# Let's move (1, 1) um with scan at 90-degree
exp.execute('move_probe', value=[[1e-6, 1e-6], 90], v0=v0)

# and in sequential operations:
action_list = [
    ['Stop', None, None],
    ['move_probe', [[1e-6, 1e-6], 90], {'v0':v0, 's0':s0}]
]
```

Here we have the option to put the two positional args of move_probe (r, angle) in one list, and still have the freedom to pass the keyword args after that.


The remaining context of this tutorial will be on how to construct workflows with **aespm.Experiment** object.

# Simple workflows 

I will use an example workflow of grid search of large ferroelectric sample to demonstrate how aespm works.

Here is the basic workflow:

1. Move the stage to the next grid location.
2. Tube the probe and then start a DART scan.
3. Move the probe to the five locations to take DART SS spectra.
4. Move the stage to the next grid location and repeat.

## Initiate the experiment object

```Python
import aespm as ae

# Define the data saving folder
save_folder = r"C:\Users\Asylum User\Documents\Asylum Research Data\240410\Sample_Name"

# initialize the Experiment object
exp = ae.Experiment(folder=save_folder)
```

## Add custom functions to exp

Let's first define the custom functions for this workflow:

```Python
# Add custom functions to the experiment object

# Function to read meter
def read_meter(self, connection=None,):
    '''
    This is a read meter function.
    '''
    ae.write_spm(commands="GetMeter()", connection=self.connection)
    w = ae.ibw_read(r"C:\Users\Asylum User\Documents\buffer\Meter.ibw", connection=self.connection)
    return w

exp.add_func(read_meter)

# Function to set the deflection 
def set_deflection(self, defl=0.5):
    
    # Zero PD
    self.execute(action='Stop')
    self.execute(action='ZeroPD', wait=3)
    defl_read = self.read_meter()[1]
    dart_init2 = [
        ['SetpointDefl', defl_read+defl, None],
        ['DARTTrigger', defl_read+defl, None],
    ]
    self.execute_sequence(operation=dart_init2)

exp.add_func(set_deflection)

# Function to move the stage with the given displacement 
def stage(self, distance):
    
    # Enable the stage move --> 5 sec, 8 seconds for safety
    self.execute('EnableStage', wait=8)
    ae.move_stage(distance=distance)
    time.sleep(2* 2)
    self.execute('DisableStage', wait=1)
    # Approaching to the sample surface --> 20 sec, let's use 30 seconds for safety
    self.execute('StartApproach', wait=45)

exp.add_func(stage)

# Function to move the probe with the given displacement 
def move_probe(self, distance, v0=None, s=None):
    
    # Enable the stage move --> 5 sec, 8 seconds for safety
    ae.move_tip(distance, v0=v0, s=s, connection=self.connection)

exp.add_func(move_probe)

# Function to check the file number in a given folder
def check_files(self):
    return ae.check_file_number(path=self.folder)
exp.add_func(check_files)

# Function to start and save an AC scan
def start_dart_scan(self, fname):
    dart_scan = [
        ['ChangeName', fname, None], # Change file names
        ['Capture', None, None], # Capture an optical image
        ['DownScan', None, 1.5], # Start a down scan
        ['check_files', None, None], # Check file numbers in the data save folder
    ]
    self.execute_sequence(operation=dart_scan)

exp.add_func(start_dart_scan)

# Function to start and save a DART spec at position [x,y]
def start_dart_ss(self, r, fname):
    
    dart_ss = [
        ['ChangeName', fname, None], # Change file names
        ['ClearForce', None, None], # Clear any existing force points
        ['GoThere', None, 1], # Move to the center of the image
        ['move_tip', [r], None, {'v0': self.param['v0'], 's': self.param['s']}], # Move the tip to location r
        ['IVDoItDART', None, None], # Start a DART spec
        ['check_files', None, 1], # Check file numbers in the data save folder
    ]
    self.execute_sequence(operation=dart_ss)

exp.add_func(start_dart_ss)
```

## Exploration

Let's store some useful parameters in the **exp.param**

```Python
# You can directly add it using exp.update_param()
exp.update_param(key='f_dart', value=353.125e3) # DART will tune the probe around this freq to make sure it can relibly track the resonance

# You can also update multiple parameters together:
p = {
    'v_dart': 1, # unit is V
    'f_width': 10e3, # Hz 
    'ScanSize': 5e-6, #um
    'ScanPixels': 256, # pixels
}
for key in p:
    exp.update_param(key=key, value=p[key])
```

```Python
num_points = 20
stepx = 10e-2 / num_points
stepy = 1e-3
displacement = [stepx, stepy]

tunes = []

# Map size
xsize, ysize = 2.5e-6, 2.5e-6
pos0 = np.array([xsize/2, ysize/2])

exp.execute('ScanSize', xsize)
exp.execute('Pixels', value=256)

for i in range(num_points):
    print("Working on Location: {}/{}".format(i+1, num_points), end='\r')
   
    # Skip the first point
    if i:
        # Move the stage to the next grid point
        exe.execute('stage', value=[displacement])

    # AC scan
    exe.execute('start_ac_scan', value='Sample_AC{:03}_'.format(i+1))
    
    # Change to DART mode
    exp.execute_sequence(operation=dart_init)
    
    # Tune the probe for two times
    w = ae.tune_probe(num=2, out=True, center=350e3)
    tunes.append(w)
    
    # Set the new deflection
    exp.execute(action='set_deflection', value=0.5)
    
    # DART scan
    exe.execute('start_dart_scan', value='Sample_DART{:03}_'.format(i+1))

    # DART Spec
    exe.execute('dart_ss', value='Sample_SS{:03}_'.format(i+1))

    # Change the mode back to the AC mode
    exp.execute_sequence(operation=ac_init)
    
np.savez('Tunes.npz', data=np.array(tunes))
```

# Complicated workflows interfacing Machine learning algorithms

# Control from remote supercomputer

First, set up a server on the local desktop that connects to the SPM controller:

Start a new notebook and run the following codes on your **local** computer:

```Python
import aespm as ae
host = "your_ip_address"
aespm.utils.connect(host=host)
```

On the **remote** server, you only need the following information to build the connection:

```Python
# host = 'IP_address_local_desktop'
# username = 'your_login_name'
# password = 'your_login_credential'

exp = ae.Experiment(folder=folder, connection=[host, username, password])
```

Then all your local notebooks should run automatically with this exp object.